In [ ]:
%cd /home/jaeheonshim/music-vibes

/home/jaeheonshim/music-vibes


In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Subset
import librosa
from tqdm import tqdm
from torchvggish import vggish_input

from vibenet.dataset import FMAWaveformDataset
from vibenet.models.teacher import PANNsMLP
from vibenet import labels

In [ ]:
device = 'cpu'

model = PANNsMLP()
model = model.to(device)
data = torch.load('checkpoints/pretrained_PANN.pt')
model.load_state_dict(data['state_dict'])

<All keys matched successfully>

In [ ]:
def eval(y):
    model.eval()

    with torch.inference_mode():
        pred = model(torch.tensor(y).unsqueeze(0).to(device))
        
    pred['acousticness'] = F.sigmoid(pred['acousticness'])
    pred['instrumentalness'] = F.sigmoid(pred['instrumentalness'])
    pred['liveness'] = F.sigmoid(pred['liveness'])
    pred['speechiness'] = torch.clamp(pred['speechiness'], 0, 1)
    pred['danceability'] = torch.clamp(pred['danceability'], 0, 1)
    pred['energy'] = torch.clamp(pred['energy'], 0, 1)
    pred['valence'] = torch.clamp(pred['valence'], 0, 1)

    return pred

In [ ]:
y, sr = librosa.load("16 Imagine Dragons - Believer.mp3", sr=32000)

pred = eval(y)

for i, label in enumerate(labels):
    print(f"{label:<20}{pred[label].item()}")

acousticness        0.47241440415382385
danceability        0.5090346336364746
energy              0.7003660202026367
instrumentalness    0.19107572734355927
liveness            0.19258780777454376
speechiness         0.5276971459388733
valence             0.5537452697753906
